In [10]:
pip install torch pandas

In [3]:
pip install mistral_inference mistral_common

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 704.9/704.9 kB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.4/88.4 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.8/394.8 kB 32.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 45.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.

In [4]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [5]:
# Download from Hugging Face
from huggingface_hub import snapshot_download
from pathlib import Path
# Define the path to save the model
mistral_models_path = Path.home().joinpath('mistral_models', '7B-Instruct-v0.3')
mistral_models_path.mkdir(parents=True, exist_ok=True)
# Download the model
snapshot_download(repo_id="mistralai/Mistral-7B-Instruct-v0.3",
                  allow_patterns=["params.json", "consolidated.safetensors", "tokenizer.model.v3"],
                  local_dir=mistral_models_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

params.json:   0%|          | 0.00/202 [00:00<?, ?B/s]

tokenizer.model.v3:   0%|          | 0.00/587k [00:00<?, ?B/s]

consolidated.safetensors:   0%|          | 0.00/14.5G [00:00<?, ?B/s]

'/root/mistral_models/7B-Instruct-v0.3'

In [7]:
import os
import pandas as pd
from pathlib import Path
from mistral_common.tokens.tokenizers.mistral import MistralTokenizer
from mistral_inference.model import Transformer
from mistral_inference.generate import generate
from mistral_common.protocol.instruct.messages import UserMessage
from mistral_common.protocol.instruct.request import ChatCompletionRequest
import torch
import urllib.request
from transformers import logging

tokenizer = MistralTokenizer.from_file(f"{mistral_models_path}/tokenizer.model.v3")
model = Transformer.from_folder(mistral_models_path)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [18]:
logging.set_verbosity_error()

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Downloaded datasets to /root/datasets


Transformer(
  (tok_embeddings): Embedding(32768, 4096)
  (norm): RMSNorm()
  (output): Linear(in_features=4096, out_features=32768, bias=False)
  (layers): ModuleDict(
    (0): TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=4096, out_features=4096, bias=False)
        (wk): Linear(in_features=4096, out_features=1024, bias=False)
        (wv): Linear(in_features=4096, out_features=1024, bias=False)
        (wo): Linear(in_features=4096, out_features=4096, bias=False)
      )
      (attention_norm): RMSNorm()
      (ffn_norm): RMSNorm()
      (feed_forward): FeedForward(
        (w1): Linear(in_features=4096, out_features=14336, bias=False)
        (w2): Linear(in_features=14336, out_features=4096, bias=False)
        (w3): Linear(in_features=4096, out_features=14336, bias=False)
      )
    )
    (1): TransformerBlock(
      (attention): Attention(
        (wq): Linear(in_features=4096, out_features=4096, bias=False)
        (wk): Linear(in_features=40

In [1]:
def generate_prompt(metadata, files_info):
    prompt = "You are given a dataset with the following information:\n\n"

    if 'title' in metadata and metadata['title']:
        prompt += f"Dataset Title: {metadata['title']}\n"
    else:
        prompt += "Dataset Title: Not available\n"

    if 'description' in metadata and metadata['description']:
        prompt += f"Description: {metadata['description']}\n"
    else:
        prompt += "Description: Not available\n"

    if 'keywords' in metadata and metadata['keywords']:
        prompt += f"Keywords: {', '.join(metadata['keywords'])}\n"
    else:
        prompt += "Keywords: Not available\n"

    if 'licenses' in metadata and metadata['licenses']:
        prompt += f"License: {metadata['licenses'][0]['name']}\n"
    else:
        prompt += "License: Not available\n"

    prompt += "\nDataset Contents:\n"
    for file_info in files_info:
        if 'filename' in file_info and file_info['filename']:
            prompt += f"File: {file_info['filename']}\n"
        else:
            prompt += "File: Not available\n"

        if 'column_names' in file_info and file_info['column_names']:
            prompt += f"- Columns: {', '.join(file_info['column_names'])}\n"
        else:
            prompt += "- Columns: Not available\n"

        if 'num_rows' in file_info:
            prompt += f"- Number of Rows: {file_info['num_rows']}\n"
        else:
            prompt += "- Number of Rows: Not available\n"

        if 'num_columns' in file_info:
            prompt += f"- Number of Columns: {file_info['num_columns']}\n"
        else:
            prompt += "- Number of Columns: Not available\n"

        if 'sample_data' in file_info and file_info['sample_data']:
            prompt += "\nSample Data:\n"
            sample_data = "\n".join([",".join(map(str, row)) for row in file_info['sample_data'][:5]])
            prompt += sample_data + "\n"
        else:
            prompt += "Sample Data: Not available\n"

        prompt += "\n"

    prompt += ("Based on the above information, please generate a summary for the dataset. "
               "The summary should include the following details:\n"
               "1. What the data is about\n"
               "2. How this data could be used\n")

    return prompt

metadata_ex = {
    "title": "Hotel Recommendation Dataset",
    "description": "",
    "keywords": ["hotels and accommodations"],
    "licenses": [{"name": "unknown"}]
}

files_info_ex = [
    {
        "filename": "train.csv",
        "column_names": ["site_name", "posa_continent", "user_location_country", "user_location_region", "user_location_city", "user_id", "is_mobile", "is_package", "channel", "srch_adults_cnt", "srch_children_cnt", "srch_rm_cnt", "srch_destination_id", "srch_destination_type_id", "is_booking", "cnt", "hotel_continent", "hotel_country", "hotel_market", "hotel_cluster"],
        "num_rows": 5,
        "num_columns": 20,
        "sample_data": [
            [2, 3, 66, 348, 48862, 12, 0, 1, 9, 2, 0, 1, 8250, 1, 0, 3, 2, 50, 628, 1],
            [2, 3, 66, 348, 48862, 12, 0, 1, 9, 2, 0, 1, 8250, 1, 1, 1, 2, 50, 628, 1],
            [2, 3, 66, 348, 48862, 12, 0, 0, 9, 2, 0, 1, 8250, 1, 0, 1, 2, 50, 628, 1],
            [2, 3, 66, 442, 35390, 93, 0, 0, 3, 2, 0, 1, 14984, 1, 0, 1, 2, 50, 1457, 80],
            [2, 3, 66, 442, 35390, 93, 0, 0, 3, 2, 0, 1, 14984, 1, 0, 1, 2, 50, 1457, 21]
        ]
    }
]

prompt = generate_prompt(metadata_ex, files_info_ex)
print(prompt)

You are given a dataset with the following information:

Dataset Title: Hotel Recommendation Dataset
Description: Not available
Keywords: hotels and accommodations
License: unknown

Dataset Contents:
File: train.csv
- Columns: site_name, posa_continent, user_location_country, user_location_region, user_location_city, user_id, is_mobile, is_package, channel, srch_adults_cnt, srch_children_cnt, srch_rm_cnt, srch_destination_id, srch_destination_type_id, is_booking, cnt, hotel_continent, hotel_country, hotel_market, hotel_cluster
- Number of Rows: 5
- Number of Columns: 20

Sample Data:
2,3,66,348,48862,12,0,1,9,2,0,1,8250,1,0,3,2,50,628,1
2,3,66,348,48862,12,0,1,9,2,0,1,8250,1,1,1,2,50,628,1
2,3,66,348,48862,12,0,0,9,2,0,1,8250,1,0,1,2,50,628,1
2,3,66,442,35390,93,0,0,3,2,0,1,14984,1,0,1,2,50,1457,80
2,3,66,442,35390,93,0,0,3,2,0,1,14984,1,0,1,2,50,1457,21

Based on the above information, please generate a summary for the dataset. The summary should include the following details:
1. What